In [1]:
# Install python Libraries to connect to MySQL hosted on GCP
!pip install pymysql
!pip install cloud-sql-python-connector[pymysql]

In [2]:
import pymysql
import pandas as pd
import numpy as np
from google.cloud.sql.connector import Connector
import sqlalchemy
import datetime
from sqlalchemy.exc import OperationalError,ProgrammingError,IntegrityError

In [3]:
# The Cloud SQL Python Connector can be used along with SQLAlchemy using the
# 'creator' argument to 'create_engine'
connector = Connector()
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        "my-project-olam-95:us-central1:my-case-study-1",
        "pymysql",
        user="root",
        password="uIF2eu1AKgIlqHNv",
        db="nifty-data",
    )
    return conn

pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

# Connect to MySQL Db
db = pool.connect()

# Create Table Statements based on Schema design discussed in task 1
create_ticker_info = "CREATE TABLE ticker_info ( symbol VARCHAR(20) PRIMARY KEY, name VARCHAR(50), industry VARCHAR(40), duration_in_nifty50 INT);"

create_nifty50_monthly = "CREATE TABLE nifty50_monthly ( symbol VARCHAR(20),report_date DATE,equity_capital BIGINT,free_float_market_cap DECIMAL,weightage DECIMAL,beta DECIMAL,R2 DECIMAL,volatility_per DECIMAL,monthly_return DECIMAL,avg_impact_cost_percent DECIMAL,PRIMARY KEY (report_date, symbol),FOREIGN KEY (symbol) REFERENCES ticker_info(symbol));"

# Create Tables
try:
    db.execute(create_ticker_info)
    db.execute(create_nifty50_monthly)
except (OperationalError, ProgrammingError) as err:
    print(err.args[0])
else:
    print("OK")

(pymysql.err.OperationalError) (1050, "Table 'ticker_info' already exists")


In [4]:
# Read Data as pandas Data Frame from given excel
# Reading date as string as deafult read for dates converst to numoy datetime which is not convenient to calculate month/year deltas
df = pd.read_excel("C:/Users/vidus/Downloads/nifty_data.xlsx",dtype={'report_date':str}) 

#Rename columns to remove spaces in column names ( easier for data understanding and operations)
df=df.rename(columns = {'Equity Capital (In Rs.)':'equity_capital','Free Float Market Capitalisation (Rs. Crores)': 'free_float_market_cap','Weightage ()':'weightage','Monthly Return':'monthly_return',})
df.columns= df.columns.str.lower()


In [5]:
# Drop rows that are absolute duplicates (There were 100 instead of 50 values for 2018-01-01 )
# Can be confirmed by running df.report_date.value_counts()
df = df.drop_duplicates()

In [6]:
# Sorting values based on report_date and symbol as we need to split data into 2 sets for above mentioned 2 tables,
# this makes it easier for many other operations as well as visualization
df.sort_values(by=['report_date','symbol'])

,symbol,name,industry,equity_capital,free_float_market_cap,weightage,beta,r2,volatility_per,monthly_return,avg_impact_cost_percent,report_date
6300,ABB,Electrical Equipment,NaN,423816750,23732.00,0.80,0.86,0.48,3.14,-25.94,0.15,2008-01-01 00:00:00
6301,ACC,Cement & Cement Products,NaN,1876243140,14411.00,0.49,0.89,0.36,3.73,-25.05,0.17,2008-01-01 00:00:00
6312,AMBUJACEM,Cement & Cement Products,NaN,3043782964,18377.00,0.62,0.46,0.17,2.25,-17.89,0.21,2008-01-01 00:00:00
6302,BAJAJAUTO,Automobiles - 2 and 3 wheelers,NaN,1011835100,23870.00,0.80,0.64,0.25,4.75,-10.23,0.17,2008-01-01 00:00:00
6303,BHARTIARTL,Telecommunication - Services,NaN,18978832460,163000.00,5.49,0.89,0.42,3.08,-13.8,0.15,2008-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3544,UPL,UPL Ltd.,PESTICIDES AND AGROCHEMICALS,1528028010,33222.69,0.70,0.91,0.18,1.72,7.24,0.03,2019-09-01 00:00:00
3546,VEDL,Vedanta Ltd.,MINING,3717196639,28077.29,0.60,1.39,0.24,3.20,10.78,0.03,2019-09-01 00:00:00
3547,WIPRO,Wipro Ltd.,COMPUTERS - SOFTWARE,11424565304,35614.94,0.76,0.34,0.05,1.28,-5.74,0.02,2019-09-01 00:00:00
3548,YESBANK,Yes Bank Ltd.,BANKS,5100619862,8657.79,0.18,1.48,0.08,6.33,-30.94,0.09,2019-09-01 00:00:00


In [7]:
# monthy return was read as str due to atleast 1 value being just a hyphen with no number after it
# Hence replacing with nan and converting to float
df['monthly_return'] = df['monthly_return'].replace('-',np.nan).astype(float)
# Changing data type for report_date to datetime.
df['report_date'] = pd.to_datetime(df['report_date'])
df.dtypes

symbol                             object
name                               object
industry                           object
equity_capital                      int64
free_float_market_cap             float64
weightage                         float64
beta                              float64
r2                                float64
volatility_per                    float64
monthly_return                    float64
avg_impact_cost_percent           float64
report_date                datetime64[ns]
dtype: object

In [8]:
# Check for Null Values in all rows
df.isnull().sum()

symbol                       0
name                        50
industry                   600
equity_capital               0
free_float_market_cap        0
weightage                    0
beta                         0
r2                           0
volatility_per               0
monthly_return               3
avg_impact_cost_percent      0
report_date                  0
dtype: int64

In [9]:
# Fill Null cells with valid values

# Setting missing numeric value to 0
df.monthly_return.fillna(0,inplace=True)

## Create data frame for Ticker values (Table ticker_info)

In [10]:
# Selecting columns "symbol","name","industry","report_date"
ticker = df.iloc[:, [0,1,2,11]]

In [11]:
ticker.sort_values(by=["symbol","name","industry"],na_position='last') # to ensure nan values are kept at last.

# Setting industry as name if industry is null, and name as symbol if name is null.
ticker["name"].fillna(ticker["symbol"], inplace=True)
ticker["industry"].fillna(ticker["name"], inplace=True)


C:\Users\vidus\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\vidus\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\vidus\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [12]:
# Grouping by symbol and countiing on unique dates to find duration in months for which a stock was in Nifty50
ticker_info = ticker.groupby('symbol')['report_date'].nunique().reset_index()
ticker_info.rename(columns = {'report_date':'duration_in_nifty50'}, inplace = True)
ticker_info

,symbol,duration_in_nifty50
0,ABB,31
1,ACC,111
2,ADANIPORTS,48
3,AMBUJACEM,118
4,ASIANPAINT,87
...,...,...
88,VEDL,40
89,VSNL,1
90,WIPRO,131
91,YESBANK,54


In [13]:
# Removing duplicates for symbols - since we have already calculated duration, we do not need report_date anymore
ticker=ticker.drop_duplicates(subset=['symbol'], keep='first', inplace=False)
ticker.drop("report_date", axis=1, inplace=True)
ticker

,symbol,name,industry
0,ABB,None,None
1,ACC,None,None
2,AMBUJACEM,None,None
3,AXISBANK,None,None
4,BHARTIARTL,None,None
...,...,...,...
3533,NESTLEIND,None,None
6302,BAJAJAUTO,None,None
6310,GLAXO,None,None
6334,REL,None,None


In [14]:
# Join Ticker and ticker info - Unique values of ticker (symbols) in excel + duration in months
ticker_info=ticker_info.set_index('symbol').join(ticker.set_index('symbol'))

In [15]:
# Reset index to level the data frame and remove symbol as index
ticker_info=ticker_info.reset_index()

In [16]:
ticker_info

,symbol,duration_in_nifty50,name,industry
0,ABB,31,None,None
1,ACC,111,None,None
2,ADANIPORTS,48,None,None
3,AMBUJACEM,118,None,None
4,ASIANPAINT,87,None,None
...,...,...,...,...
88,VEDL,40,None,None
89,VSNL,1,None,None
90,WIPRO,131,None,None
91,YESBANK,54,None,None


In [17]:
# Bulk add ticker_info to mySQL
# (Sinec we already have a table, we use append mode, default is fail if table exists. index is set to false, defaukt is True)
try:
    ticker_info.to_sql("ticker_info", db, if_exists='append',index=False)
except (OperationalError, ProgrammingError, IntegrityError) as err:
    print(err.args[0])

(pymysql.err.IntegrityError) (1062, "Duplicate entry 'ABB' for key 'PRIMARY'")


In [18]:
# Bulk add nifty50_monthly required columns to mySQL from df
try:
    df.iloc[:, [0,11,3,4,5,6,7,8,9,10]].to_sql("nifty50_monthly", db, if_exists='append', index=False)
except (OperationalError, ProgrammingError, IntegrityError) as err:
    print(err.args[0])

(pymysql.err.IntegrityError) (1062, "Duplicate entry '2009-08-01-ABB' for key 'PRIMARY'")


In [19]:
pd.read_sql_query("select * from ticker_info", pool.connect())

,symbol,name,industry,duration_in_nifty50
0,ABB,ABB Ltd.,ELECTRICAL EQUIPMENT,31
1,ACC,ACC Ltd.,CEMENT AND CEMENT PRODUCTS,111
2,ADANIPORTS,Adani Ports and Special Economic Zone Ltd.,SHIPPING,48
3,AMBUJACEM,Ambuja Cements Ltd.,CEMENT AND CEMENT PRODUCTS,118
4,ASIANPAINT,Asian Paints Ltd.,PAINTS,87
...,...,...,...,...
88,VEDL,Vedanta Ltd.,MINING,40
89,VSNL,Telecommunication - Services,None,1
90,WIPRO,Wipro Ltd.,COMPUTERS - SOFTWARE,131
91,YESBANK,Yes Bank Ltd.,BANKS,54


In [20]:
pd.read_sql_query("select * from nifty50_monthly", pool.connect())

,symbol,report_date,equity_capital,free_float_market_cap,weightage,beta,R2,volatility_per,monthly_return,avg_impact_cost_percent
0,ABB,2008-01-01,423816750,23732.0,1.0,1.0,0.0,3.0,-26.0,0.0
1,ACC,2008-01-01,1876243140,14411.0,0.0,1.0,0.0,4.0,-25.0,0.0
2,AMBUJACEM,2008-01-01,3043782964,18377.0,1.0,0.0,0.0,2.0,-18.0,0.0
3,BAJAJAUTO,2008-01-01,1011835100,23870.0,1.0,1.0,0.0,5.0,-10.0,0.0
4,BHARTIARTL,2008-01-01,18978832460,163000.0,5.0,1.0,0.0,3.0,-14.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6795,UPL,2019-09-01,1528028010,33223.0,1.0,1.0,0.0,2.0,7.0,0.0
6796,VEDL,2019-09-01,3717196639,28077.0,1.0,1.0,0.0,3.0,11.0,0.0
6797,WIPRO,2019-09-01,11424565304,35615.0,1.0,0.0,0.0,1.0,-6.0,0.0
6798,YESBANK,2019-09-01,5100619862,8658.0,0.0,1.0,0.0,6.0,-31.0,0.0


## TASK 1 - Data Retrieval 
### Write query to get year wise number of stocks per Industry. 


In [21]:
result1 = pd.read_sql_query("SELECT ticker_info.industry, YEAR(nifty50_monthly.report_date) AS year, count(distinct nifty50_monthly.Symbol) FROM nifty50_monthly LEFT JOIN ticker_info on nifty50_monthly.Symbol = ticker_info.Symbol group by ticker_info.Industry, year ORDER BY year,industry;", db)

In [22]:
result1

,industry,year,count(distinct nifty50_monthly.Symbol)
0,None,2008,4
1,ALUMINIUM,2008,2
2,AUTOMOBILES - 2 AND 3 WHEELERS,2008,1
3,AUTOMOBILES - 4 WHEELERS,2008,3
4,BANKS,2008,4
...,...,...,...
282,REFINERIES,2019,4
283,SHIPPING,2019,1
284,STEEL AND STEEL PRODUCTS,2019,2
285,TELECOMMUNICATION - EQUIPMENT,2019,1


### Stocks that were part of index for longest and shortest duration.

In [23]:
result2 = pd.read_sql_query("SELECT * FROM ticker_info WHERE duration_in_nifty50 in (SELECT MIN(duration_in_nifty50) FROM ticker_info UNION SELECT MAX(duration_in_nifty50) FROM ticker_info) ORDER BY duration_in_nifty50 ASC;",db)

In [24]:
max_dur = result2.duration_in_nifty50.max()

In [25]:
min_dur = result2.duration_in_nifty50.min()

In [26]:
max_duration_stocks = result2[result2['duration_in_nifty50']==max_dur]['symbol'].tolist()
min_duration_stocks = result2[result2['duration_in_nifty50']==min_dur]['symbol'].tolist()

In [27]:
min_duration_stocks

['NESTLEIND', 'VSNL']

In [28]:
max_duration_stocks

['BHARTIARTL',
 'BPCL',
 'CIPLA',
 'GAIL',
 'HCLTECH',
 'HDFC',
 'HDFCBANK',
 'HINDALCO',
 'HINDUNILVR',
 'ICICIBANK',
 'ITC',
 'LT',
 'M&M',
 'MARUTI',
 'NTPC',
 'ONGC',
 'RELIANCE',
 'SBIN',
 'SUNPHARMA',
 'TATAMOTORS',
 'TATASTEEL',
 'TCS']

### TASK 2 - 7 Calculate year wise high and low Equity Capital for each stock. 

In [29]:
result3 = pd.read_sql_query("SELECT symbol, YEAR(report_date) AS year, min(equity_capital), max(equity_capital) as max_equity_capital FROM nifty50_monthly group by symbol, year ORDER BY symbol,year;", db)

In [30]:
result3

,symbol,year,min(equity_capital),max_equity_capital
0,ABB,2008,423816750,423816750
1,ABB,2009,423816750,423816750
2,ABB,2010,423816750,423816750
3,ACC,2008,1876243140,1876608800
4,ACC,2009,1876817290,1877109270
...,...,...,...,...
644,ZEEL,2015,960448720,960448720
645,ZEEL,2016,960448720,960448720
646,ZEEL,2017,960448720,960453620
647,ZEEL,2018,960453620,960466500
